In [ ]:
import polars as pl
import os

import sys
sys.path.append('..')
from src import PlottingFunctions

plotter = PlottingFunctions.Plotter()
import numpy as np
import json

from skimage import measure
from skimage.measure import label

from src import IOFunctions; IO = IOFunctions.IO_Functions()

import matplotlib.pyplot as plt

from src import AnalysisFunctions; A_F = AnalysisFunctions.Analysis_Functions()

from src import CoincidenceFunctions; C_F = CoincidenceFunctions.Coincidence_Functions()

In [ ]:
cingulate_PD_region = r'S:\ASAP_Imaging_Data\Main_Survey\20241105_oligomers_in_cells\cingulate\microglia\PD_analysis'
frontal_PD_region = r'S:\ASAP_Imaging_Data\Main_Survey\20241105_oligomers_in_cells\frontal\microglia\PD_analysis'

In [ ]:
def file_selector(folder, oligomer_data, percentile=20):
    files = os.listdir(folder)
    file = np.sort([x for x in files if 'percentile' in x and '_'+str(int(percentile))+'_' in x and '3Danalysis' in x])[0]
    data = pl.read_csv(os.path.join(folder, file))
    photonthreshold = float(file.split('_photonthreshold_')[1].split('_photons_')[0].replace('p', '.'))
    oligomer_data = oligomer_data.filter(pl.col('sum_intensity_in_photons') < 3000)
    percentile = oligomer_data['sum_intensity_in_photons'].rank()/len(oligomer_data['sum_intensity_in_photons'])
    oligomer_data = oligomer_data.with_columns(percentile.alias("alpha"))
    return oligomer_data

In [ ]:
cingulate_PD_data = pl.read_csv(os.path.join(cingulate_PD_region, 'spot_analysis.csv'))
frontal_PD_data = pl.read_csv(os.path.join(frontal_PD_region, 'spot_analysis.csv'))
cingulate_PD_data = file_selector(cingulate_PD_region, cingulate_PD_data, percentile=90)
frontal_PD_data = file_selector(frontal_PD_region, frontal_PD_data, percentile=90)

In [ ]:
overall_data = pl.concat([cingulate_PD_data, frontal_PD_data])

In [ ]:
del frontal_PD_data
del cingulate_PD_data

In [ ]:
files = ['Position 1 [9]_XY1697740734_Z00_T4', 'Position 1 [9]_XY1697721826_Z00_T5', 'Position 1 [9]_XY1697643793_Z00_T5']
filenames = []
for fname in files:
    filenames = np.hstack([filenames, np.unique([x for x in overall_data['image_filename'].to_numpy() if fname in x])])

In [ ]:
def quick_plotter(filename, protein_string='C1',
                                       cell_string='C0', xmin=0, 
                                       width=1200, ymin=0, zplane=None, labelval=None):
    raw_protein_image = IO.read_tiff_tophotons(filename)
    analysis_folder = os.path.split(os.path.split(filename)[0])[0]+"_analysis"

    cell_filename = filename.split(protein_string+'.tif')[0]+cell_string+'.tif'
    raw_cell_image = IO.read_tiff_tophotons(cell_filename)
    cell_MIP = np.max(raw_cell_image, axis=0)
    if zplane is not None:
        protein_MIP = raw_protein_image[zplane, : , :]
    else:
        protein_MIP = np.max(raw_protein_image, axis=0)
    cell_mask_string = os.path.split(filename)[-1].split(protein_string+'.tif')[0]+cell_string+'_cellMask_cleanedup.tiff'
    cell_mask = IO.read_tiff(os.path.join(analysis_folder, cell_mask_string))
    labeled_image = label(
        cell_mask, connectivity=3, return_num=False
    )
    mask_toplot = np.max(labeled_image, axis=0)
    if labelval is not None:
        fig, axs = plotter.one_column_plot()
        mask_toplot = np.max(labeled_image, axis=0)
        mask_toplot[mask_toplot != labelval] = 0
        mask_toplot[mask_toplot == labelval] = 1
        axs[0] = plotter.image_plot(axs=axs[0], data=protein_MIP[xmin:xmin+width, ymin:ymin+width],
                                            cbar='off', scalebarsize=200, scalebarlabel='200 nm')      
    
        axs[0].contour(mask_toplot[xmin:xmin+width, ymin:ymin+width], [0.5], lw=0.75, colors="white")
        
        plt.tight_layout()
        plt.show(block=False)
    else:
        for value in np.arange(len(np.unique(mask_toplot))):
            fig, axs = plotter.one_column_plot()
            mask_toplot = np.max(labeled_image, axis=0)
            mask_toplot[mask_toplot != value] = 0
            mask_toplot[mask_toplot == value] = 1
            axs[0] = plotter.image_plot(axs=axs[0], data=protein_MIP[xmin:xmin+width, ymin:ymin+width],
                                                cbar='off', scalebarsize=200, scalebarlabel='200 nm')      
        
            axs[0].contour(mask_toplot[xmin:xmin+width, ymin:ymin+width], [0.5], lw=0.75, colors="white")
            
            plt.tight_layout()
            plt.show(block=False)
    return

In [ ]:
quick_plotter(filenames[1], xmin=382, width=20, ymin=422, zplane=21, labelval=4)

In [ ]:
labels = np.array([2, 4, 6])
oligomer_zs = np.array([22, 21,20])
oligomers_xmins = np.array([483,499,846])
oligomers_ymins = np.array([513,623,729])
oligomer_widths = np.array([20, 20, 20])

In [ ]:
oligomers2_xmins = np.array([536, 382, 721])
oligomers2_ymins = np.array([523,422,772])


In [ ]:
xmins = np.array([255, 250, 520])
widths = np.array([415, 450, 400])
ymins = np.array([350, 230, 560])

In [ ]:
def cell_oligomer_3panel_generator_rawoligomer(filenames, oligomer_data, folder_to_save, protein_string='C1',
                                       cell_string='C0', xmins=0, 
                                       ymins=0, widths=1200, zvals=22):
    mm = 1/25.4
    fig, axs = plotter.two_column_plot(ncolumns=3, widthratio=[1,1,1], height=(170/3)*mm, width=180*mm)

    for i in np.arange(3):
        filename = filenames[i]
        xmin = xmins[i]
        ymin = ymins[i]
        width = widths[i]
        analysis_folder = os.path.split(os.path.split(filename)[0])[0]+"_analysis"
    
        protein_data = IO.read_tiff_tophotons(filename)[zvals[i], :, :]
        axs[i] = plotter.image_plot(axs=axs[i], data=protein_data[xmin:xmin+width, ymin:ymin+width],
                                            cbar='off', masklinewidth=0.5, scalebarlabel='300 nm', scalebarsize=300)        
    return fig, axs

In [ ]:
folder_to_save = r'C:\Users\jsb92\Cambridge University Dropbox\Joseph Beckwith'
fig, axs = cell_oligomer_3panel_generator_rawoligomer(filenames, overall_data, folder_to_save, xmins=oligomers2_xmins, 
                                                      ymins=oligomers2_ymins, widths=oligomer_widths, zvals=oligomer_zs)
plt.tight_layout()
plt.savefig(os.path.join(folder_to_save, 'Example_oligomersinmicroglia2.svg'), format='svg', dpi=1200)

In [ ]:
def cell_oligomer_3panel_generator(filenames, oligomer_data, folder_to_save, protein_string='C1',
                                       cell_string='C0', xmins=0, 
                                       ymins=0, widths=1200, labelvals=0, points=True):
    mm = 1/25.4
    fig, axs = plotter.two_column_plot(ncolumns=3, widthratio=[1,1,1], height=(170/3)*mm, width=180*mm)

    for i in np.arange(3):
        filename = filenames[i]
        xmin = xmins[i]
        ymin = ymins[i]
        width = widths[i]
        labelval = labelvals[i]
        analysis_folder = os.path.split(os.path.split(filename)[0])[0]+"_analysis"
        oligomers = oligomer_data.filter(pl.col('image_filename') == filename)
        oligomers = oligomers.filter(pl.col("alpha") > 0.9)
        x = oligomers['x'].to_numpy()
        y = oligomers['y'].to_numpy()
        alpha = oligomers['alpha'].to_numpy()
        zi = int(np.unique(oligomers['zi'].to_numpy())[0])
        zf = int(np.unique(oligomers['zf'].to_numpy())[0])
    
        cell_filename = filename.split(protein_string+'.tif')[0]+cell_string+'.tif'
        raw_cell_image = IO.read_tiff_tophotons(cell_filename)[zi-1:zf-1, :, :]
        cell_MIP = np.nanmax(raw_cell_image, axis=0)
        cell_mask_string = os.path.split(filename)[-1].split(protein_string+'.tif')[0]+cell_string+'_cellMask_cleanedup.tiff'
        cell_mask = IO.read_tiff(os.path.join(analysis_folder, cell_mask_string))
        labeled_image = label(
        cell_mask, connectivity=3, return_num=False
        )
        mask_toplot = np.max(labeled_image, axis=0)
        mask_toplot[mask_toplot != labelval] = 0
        mask_toplot[mask_toplot == labelval] = 1
        
        punctaindices = (x > xmin) & (x < xmin+width) & (y > ymin) & (y < ymin+width)
        x = x[punctaindices] 
        y = y[punctaindices]
        alpha = alpha[punctaindices]
        centroids = np.vstack([np.array(x, dtype=int), np.array(y, dtype=int)]).T
        mask_indices = A_F.generate_indices(np.array(mask_toplot, dtype=bool), (1200, 1200), is_mask=True)
        spot_indices = A_F.generate_indices(centroids, (1200, 1200))
        spot_indices = C_F._apply_blur(spot_indices, (1200, 1200), 1)
    
        in_cell = C_F.test_spot_spot_overlap(spot_indices, mask_indices, n_spot1=len(x), raw=True)
        x = x[in_cell == True]
        y = y[in_cell == True]
        alpha = alpha[in_cell == True]
        
        colours = np.array(np.full(len(x), 'red'), dtype='str')
        
        savestring =  os.path.split(filename)[-1].split(protein_string+'.tif')[0]+cell_string+'_examplemicroglia_witholigs.svg'
        if points == True:
            axs[i] = plotter.image_scatter_plot(axs=axs[i], data=cell_MIP[xmin:xmin+width, ymin:ymin+width], xdata=x-xmin, ydata=y-ymin, facecolor=colours, 
                                                alpha_scatter=alpha, s=2, lws=0, plotmask=True, mask=mask_toplot[xmin:xmin+width, ymin:ymin+width], cbar='off',
                                                masklinewidth=0.75, maskcolor='#91d1c2', scalebarlabel=r'10 $\mu$m', scalebarsize=10000)
        else:
            axs[i] = plotter.image_plot(axs=axs[i], data=cell_MIP[xmin:xmin+width, ymin:ymin+width],
                                                plotmask=True, mask=mask_toplot[xmin:xmin+width, ymin:ymin+width], cbar='off', masklinewidth=0.5)        
    return fig, axs

In [ ]:
folder_to_save = r'C:\Users\jsb92\Cambridge University Dropbox\Joseph Beckwith'
fig, axs = cell_oligomer_3panel_generator(filenames, overall_data, folder_to_save, xmins=xmins, ymins=ymins, widths=widths, labelvals=labels)
plt.tight_layout()
plt.savefig(os.path.join(folder_to_save, 'Example_microgliafigure.svg'), format='svg', dpi=1200)